# Домашнее задание № 1

Вам необходимо выбор стратегии валидации и обработку данных от ошибок заполнения.

# Часть 3. Стратегия валидации

Итак, у вас есть данные для обучения и данные, на которых необходимо сделать предсказание. Для оценки точности необходимо проводить валидацию (так как мало данных, можно проводить кросс валидацию).
Валидацию проще всего реализовать методами sklearn, например:
- KFold
- ShuffleSplit
- StratifiedKFold
- StratifiedShuffleSplit
- GroupKFold
- GroupShuffleSplit

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.model_selection import KFold, ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit, GroupKFold, GroupShuffleSplit

In [2]:
X_train = pd.read_csv('train_features_with_answers.csv')
X_train.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,MS,M,NaN,U,LE3,T,1,3,at_home,services,...,yes,yes,4,3,3,2,3,3,0,9
1,GP,M,NaN,R,LE3,T,3,3,teacher,other,...,yes,no,3,3,4,3,5,3,16,12
2,GP,M,15.0,NaN,GT3,T,3,4,other,other,...,yes,no,5,5,1,1,1,5,0,13
3,MS,M,17.0,R,GT3,T,4,3,services,other,...,yes,yes,4,5,5,1,3,2,4,11
4,GP,F,20.0,R,GT3,T,2,1,other,other,...,yes,yes,1,2,3,1,2,2,8,12


#### Примеры разбиений (раскомментить нужную строку для стратегии)

In [3]:
# CV strategy

# cv_folds = KFold(n_splits=5)
# cv_folds = ShuffleSplit(n_splits=5)
# cv_folds = StratifiedKFold(n_splits=5)
cv_folds = StratifiedShuffleSplit(n_splits=5)
# cv_folds = GroupKFold(n_splits=2)
# cv_folds = GroupShuffleSplit(n_splits=2)
'Group' not in type(cv_folds).__name__

True

In [4]:
# CV example
fold_collection = {}
# base
if 'Stratified' not in type(cv_folds).__name__ and 'Group' not in type(cv_folds).__name__:
    for i, (train_index, test_index) in enumerate(cv_folds.split(X_train)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")
        fold_collection[i] = {
            "Train": [X_train.loc[:, X_train.columns != 'G3'].iloc[train_index], X_train.loc[:, X_train.columns == 'G3'].iloc[train_index]],
            "Test": [X_train.loc[:, X_train.columns != 'G3'].iloc[test_index], X_train.loc[:, X_train.columns == 'G3'].iloc[test_index]],
        }


# stratified
if 'Stratified' in type(cv_folds).__name__:
    design, target = X_train.loc[:, X_train.columns != 'G3'], X_train['G3']
    for i, (train_index, test_index) in enumerate(cv_folds.split(design, target)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")
        fold_collection[i] = {
            "Train": [X_train.loc[:, X_train.columns != 'G3'].iloc[train_index], X_train.loc[:, X_train.columns == 'G3'].iloc[train_index]],
            "Test": [X_train.loc[:, X_train.columns != 'G3'].iloc[test_index], X_train.loc[:, X_train.columns == 'G3'].iloc[test_index]],
        }

        print('train -  {}   |   test -  {}'.format(np.bincount(target[train_index]), np.bincount(target[test_index])))
        
# group
if 'Group' in type(cv_folds).__name__:
    design, target = X_train.loc[:, X_train.columns != 'G3'], X_train['G3']
    group = design["higher"]
    for i, (train_index, test_index) in enumerate(cv_folds.split(design, target, groups = group)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")

        print("test groups")
        print(f"Train groups X: {list(X_train.loc[:, X_train.columns != 'G3'].iloc[train_index]['higher'].unique())}")
        print(f"Test groups X: {list(X_train.loc[:, X_train.columns != 'G3'].iloc[test_index]['higher'].unique())}")
        fold_collection[i] = {
            "Train": [X_train.loc[:, X_train.columns != 'G3'].iloc[train_index], X_train.loc[:, X_train.columns == 'G3'].iloc[train_index]],
            "Test": [X_train.loc[:, X_train.columns != 'G3'].iloc[test_index], X_train.loc[:, X_train.columns == 'G3'].iloc[test_index]],
        }

fold_collection

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

## Разделите данные и объясните выбор стратегии

In [5]:
default_cv = ShuffleSplit(n_splits=5)
group_cv = GroupShuffleSplit(n_splits=2)

design, target = X_train.loc[:, X_train.columns != 'G3'], X_train['G3']
group = design["higher"]

cv_indexes = {"shuffle": {}, "group": {}}
for i, (train_index, test_index) in enumerate(default_cv.split(X_train)):
    cv_indexes["shuffle"][i] = {
        "Train": [X_train.loc[:, X_train.columns != 'G3'].iloc[train_index], X_train.loc[:, X_train.columns == 'G3'].iloc[train_index]],
        "Test": [X_train.loc[:, X_train.columns != 'G3'].iloc[test_index], X_train.loc[:, X_train.columns == 'G3'].iloc[test_index]],
    }

for i, (train_index, test_index) in enumerate(cv_folds.split(design, target, groups = group)):
    cv_indexes["group"][i] = {
        "Train": [X_train.loc[:, X_train.columns != 'G3'].iloc[train_index], X_train.loc[:, X_train.columns == 'G3'].iloc[train_index]],
        "Test": [X_train.loc[:, X_train.columns != 'G3'].iloc[test_index], X_train.loc[:, X_train.columns == 'G3'].iloc[test_index]],
    }

cv_indexes

{'shuffle': {0: {'Train': [    school sex   age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
    99      GP   M  17.0       R     LE3       T     1     2     other     other   
    4       GP   F  20.0       R     GT3       T     2     1     other     other   
    450     GP   F  15.0       U     GT3       T     2     2     other     other   
    57      GP   M  17.0       U     GT3       T     3     2  services  services   
    248     GP   F  18.0       U     LE3       T     2     2   at_home  services   
    ..     ...  ..   ...     ...     ...     ...   ...   ...       ...       ...   
    423     GP   M  15.0       U     LE3       T     1     2     other   at_home   
    351     GP   M  16.0       U     GT3       T     1     2  services  services   
    259     MS   F   1.0       R     LE3       T     2     2     other     other   
    424     MS   M  15.0       R     LE3       T     1     3   at_home     other   
    81      GP   M  15.0       U     GT3       T     

__Ваш ответ:__ лучше бить по дефолту через _StratifiedShuffleSplit_ для классификации, тк мы не знаем, обьединены ли данные одной природой, чтобы использовать GroupFold, и лучше перемешать данные для генерации независимой выборки, а так сохранится доля классов в test train и будет сохраняться распределение классов. 

Данную задачу можно рассматривать и как регрессию, и как классификацию. Для регрессии сложно взять Stratified тк нету деления по классам, поэтому лучше выбрать _ShuffleSplit_

Если в данных есть какая-то группировка по природе объектов - лучше использовать Group strategy. Возможно объекты сгруппированы по Fedu, higher и studytime (высокая корреляция с G3). Поэтому можно сделать группы с высшим образованием (_higher_ - бинарный признак, лекго делить)

__Вывод:__ Стоит попробовать Group Shuffle на higher и либо StratifiedShuffleSplit (если рассматривать как классификацию), либо ShuffleSplit (если как регрессию)

# Часть 4. Заполнение пропусков в данных 

## Фильтрация по диапазону значений 

Проверьте, что все характеристики принадлежат тому диапазону значений, который приведен в описании к исходным данным. Например, если в графе "Пол" есть значение FF, P или -1, то это явная ошибка заполнения.

Приведите данные в соответствии с описанием данных. Если значение указано неверно - укажите ```NaN```.

In [6]:
for column_name in list(X_train.columns):
    print(f"Column '{column_name}':\n\tUnique values: {list(X_train[column_name].unique())}\tCount: {len(X_train[column_name].unique())}")

Column 'school':
	Unique values: ['MS', 'GP']	Count: 2
Column 'sex':
	Unique values: ['M', 'F', 'D', 'C', 'B', 'A']	Count: 6
Column 'age':
	Unique values: [nan, 15.0, 17.0, 20.0, 18.0, 16.0, 19.0, 161.0, 181.0, 151.0, 116.0, 21.0, 22.0, -1.0, 1.0, 5.0, 8.0]	Count: 17
Column 'address':
	Unique values: ['U', 'R', nan]	Count: 3
Column 'famsize':
	Unique values: ['LE3', 'GT3']	Count: 2
Column 'Pstatus':
	Unique values: ['T', 'A']	Count: 2
Column 'Medu':
	Unique values: [1, 3, 4, 2, 0]	Count: 5
Column 'Fedu':
	Unique values: [3, 4, 1, 2, 0]	Count: 5
Column 'Mjob':
	Unique values: ['at_home', 'teacher', 'other', 'services', 'health']	Count: 5
Column 'Fjob':
	Unique values: ['services', 'other', 'at_home', 'teacher', 'health']	Count: 5
Column 'reason':
	Unique values: ['course', 'home', 'reputation', 'other']	Count: 4
Column 'guardian':
	Unique values: ['mother', 'father', 'other']	Count: 3
Column 'traveltime':
	Unique values: [1, 3, 2, 4]	Count: 4
Column 'studytime':
	Unique values: [1, 2, 3

__Misvalues columns__: sex, age

In [7]:
# sex
if len(X_train["sex"].unique()) > 2:
    odd_values = list(X_train["sex"].unique())
    odd_values.remove("M")
    odd_values.remove("F")
    for odd_sex in odd_values:
        X_train["sex"] = X_train["sex"].replace(odd_sex, np.nan)

# age
X_train.loc[(X_train['age'] > 22) | (X_train['age'] < 15), 'age'] = np.nan

## Адаптивное заполнения пропусков 

Реализуйте стратегию заполнения пропусков данных на основе схожих примеров. Обучите один из известных вам алгоритмов (knn, Naive Bayes, linear regression) для предсказания той характеристики, пропуски которой присутсвуют в данных. В качестве характеристики для обучения алгоритмов __нельзя__ использовать ```G3```.

In [8]:
design = X_train.loc[:, X_train.columns != "G3"]
design = pd.get_dummies(design, columns=[elem for elem in list(design.columns) if (elem != 'address' and elem != 'sex' and elem != 'age' and elem != 'absences')])

knn for sex

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

design_notnull = design[(design['sex'].notnull()) & (design['age'].notnull()) & (design['address'].notnull())]
design_notnull.loc[(design_notnull['address'] == 'U'), 'address'] = True
design_notnull.loc[(design_notnull['address'] == 'R'), 'address'] = False
design_notnull.loc[(design_notnull['sex'] == 'M'), 'sex'] = True
design_notnull.loc[(design_notnull['sex'] == 'F'), 'sex'] = False

design_age = design_notnull.loc[:, (design_notnull.columns != "age")]
age_target = design_notnull['age']

design_age_train, design_age_test, age_target_train, age_target_test = train_test_split(design_age, age_target, test_size=0.2, random_state=42)

In [10]:
design_notnull.loc[:, (design_notnull.columns != "age")]

,sex,address,absences,school_GP,school_MS,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Medu_0,...,Walc_1,Walc_2,Walc_3,Walc_4,Walc_5,health_1,health_2,health_3,health_4,health_5
3,True,False,4,False,True,True,False,False,True,False,...,False,False,True,False,False,False,True,False,False,False
4,False,False,8,True,False,True,False,False,True,False,...,False,True,False,False,False,False,True,False,False,False
5,False,True,2,True,False,True,False,False,True,False,...,True,False,False,False,False,False,False,True,False,False
6,False,True,1,False,True,True,False,False,True,False,...,False,True,False,False,False,True,False,False,False,False
7,False,True,9,False,True,True,False,False,True,False,...,False,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,True,True,0,True,False,True,False,False,True,False,...,True,False,False,False,False,False,False,True,False,False
450,False,True,4,True,False,True,False,False,True,False,...,True,False,False,False,False,False,False,True,False,False
451,True,True,4,True,False,True,False,False,True,False,...,False,True,False,False,False,False,False,False,False,True
452,False,False,0,False,True,False,True,False,True,False,...,False,True,False,False,False,False,False,False,True,False


Simple way with **KNN Imputer**

In [11]:
from sklearn.impute import KNNImputer

design_copy = design.copy()

design_copy.loc[(design_copy['address'] == 'U'), 'address'] = True
design_copy.loc[(design_copy['address'] == 'R'), 'address'] = False
design_copy.loc[(design_copy['sex'] == 'M'), 'sex'] = True
design_copy.loc[(design_copy['sex'] == 'F'), 'sex'] = False

age_i = design_copy.columns.get_loc('age')
address_i = design_copy.columns.get_loc('address')
sex_i = design_copy.columns.get_loc('sex')

imputer = KNNImputer(n_neighbors=2)
design_fillna = imputer.fit_transform(design_copy)

X_train['age'] = design_fillna[:,age_i]
X_train['sex'] = design_fillna[:,address_i]
X_train['address'] = design_fillna[:,sex_i]

X_train.loc[(X_train['address'] == True), 'address'] = 'U'
X_train.loc[(X_train['address'] == False), 'address'] = 'R'
X_train.loc[(X_train['sex'] == True), 'sex'] = 'M'
X_train.loc[(X_train['sex'] == False), 'sex'] = 'F'

/tmp/ipykernel_23205/742170770.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'U' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.loc[(X_train['address'] == True), 'address'] = 'U'
/tmp/ipykernel_23205/742170770.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'M' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.loc[(X_train['sex'] == True), 'sex'] = 'M'


Проверка

In [12]:
X_train

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,MS,M,18.0,U,LE3,T,1,3,at_home,services,...,yes,yes,4,3,3,2,3,3,0,9
1,GP,F,17.0,U,LE3,T,3,3,teacher,other,...,yes,no,3,3,4,3,5,3,16,12
2,GP,M,15.0,U,GT3,T,3,4,other,other,...,yes,no,5,5,1,1,1,5,0,13
3,MS,F,17.0,U,GT3,T,4,3,services,other,...,yes,yes,4,5,5,1,3,2,4,11
4,GP,F,20.0,R,GT3,T,2,1,other,other,...,yes,yes,1,2,3,1,2,2,8,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,GP,M,15.0,U,GT3,T,4,2,other,other,...,yes,no,3,3,3,1,1,3,0,10
450,GP,M,15.0,R,GT3,T,2,2,other,other,...,yes,no,5,1,2,1,1,3,4,10
451,GP,M,16.0,U,GT3,T,4,4,services,services,...,yes,no,5,3,2,1,2,5,4,15
452,MS,F,15.0,R,LE3,T,1,1,at_home,services,...,yes,no,5,4,3,1,2,4,0,10


In [13]:
for column_name in list(X_train.columns):
    print(f"Column '{column_name}':\n\tUnique values: {list(X_train[column_name].unique())}\tCount: {len(X_train[column_name].unique())}")

Column 'school':
	Unique values: ['MS', 'GP']	Count: 2
Column 'sex':
	Unique values: ['M', 'F', 0.5]	Count: 3
Column 'age':
	Unique values: [18.0, 17.0, 15.0, 20.0, 16.0, 15.5, 17.5, 16.5, 19.0, 21.0, 22.0]	Count: 11
Column 'address':
	Unique values: ['U', 'R', 0.5]	Count: 3
Column 'famsize':
	Unique values: ['LE3', 'GT3']	Count: 2
Column 'Pstatus':
	Unique values: ['T', 'A']	Count: 2
Column 'Medu':
	Unique values: [1, 3, 4, 2, 0]	Count: 5
Column 'Fedu':
	Unique values: [3, 4, 1, 2, 0]	Count: 5
Column 'Mjob':
	Unique values: ['at_home', 'teacher', 'other', 'services', 'health']	Count: 5
Column 'Fjob':
	Unique values: ['services', 'other', 'at_home', 'teacher', 'health']	Count: 5
Column 'reason':
	Unique values: ['course', 'home', 'reputation', 'other']	Count: 4
Column 'guardian':
	Unique values: ['mother', 'father', 'other']	Count: 3
Column 'traveltime':
	Unique values: [1, 3, 2, 4]	Count: 4
Column 'studytime':
	Unique values: [1, 2, 3, 4]	Count: 4
Column 'failures':
	Unique values: [0